# IEOR 242 Final Project
Prepare data by reading MDA extracts from files and merging them with labels. Result is saved in a Pickle file.

In [1]:
import glob
import re
import pandas as pd
import pickle

from clean_helper import cleanup_mda

In [2]:
import pymysql
from sqlalchemy import create_engine
from sqlalchemy.dialects import mysql

# Connect to team database
engine = create_engine('mysql+pymysql://<user>:<password>@<host>[:<port>]/<dbname>')

In [3]:
# Path to MDA section files
REPORT_PATH = 'assignment-07-report-mda/*'

# Regex for parsing the file list
sec_regex = re.compile(r'mdna_(\d{4})_Q(\d{1})_(\d+)_.+_10-K_\d{4}-\d{2}-\d{2}')

# Path to the Pickle file
PICKLE_FILE = 'reports_april_25_project.pickle'

## Label Loading
Load the table with the labels.

In [4]:
# Read labels from database
fin_label_df = pd.read_sql_query('SELECT a.cik AS cik, a.fyearq AS year, a.fqtr AS quarter, a.saleq, ' \
                                 'a.oiadpq_posneg, a.atq_posneg, a.ceqq_posneg, a.saleq_posneg, a.intanq_posneg, a.ROE_posneg, ' \
                                 'a.oiadpq_change, a.atq_change, a.ceqq_change, a.saleq_change, a.intanq_change, a.ROE_change, ' \
                                 'b.gsubind AS subsector, b.conm AS name, b.tic FROM Final_Table_v3 a ' \
                                 'LEFT JOIN Compustat_sub b ON a.cik = b.cik AND a.fyearq = b.fyear', engine)

# Clean the ROE_posneg column
fin_label_df['ROE_posneg'] = fin_label_df['ROE_posneg'].map(lambda v: v.replace('\r', ''))

print('Number of reports: %d' % len(fin_label_df))
fin_label_df.head()

Number of reports: 54544


,cik,year,quarter,saleq,oiadpq_posneg,atq_posneg,ceqq_posneg,saleq_posneg,intanq_posneg,ROE_posneg,oiadpq_change,atq_change,ceqq_change,saleq_change,intanq_change,ROE_change,subsector,name,tic
0,1800,2010,1,7698.354,pos,pos,neg,pos,neg,pos,22.832,0.043,-4.546,14.648,-4.622,28.682,35101010.0,ABBOTT LABORATORIES,ABT
1,1800,2010,2,8826.014,pos,pos,pos,neg,pos,neg,6.578,7.479,7.494,-1.717,16.213,-0.852,35101010.0,ABBOTT LABORATORIES,ABT
2,1800,2010,3,8674.505,pos,pos,pos,pos,neg,pos,19.793,3.640,4.598,14.910,-1.309,14.526,35101010.0,ABBOTT LABORATORIES,ABT
3,1800,2010,4,9967.848,neg,pos,pos,neg,pos,neg,-28.107,3.681,10.012,-9.300,2.689,-34.650,35101010.0,ABBOTT LABORATORIES,ABT
4,1800,2011,1,9040.850,pos,pos,pos,pos,neg,pos,22.804,0.572,7.102,6.365,-1.180,14.661,35101010.0,ABBOTT LABORATORIES,ABT


## Reading Files & Matching
Read the file contents and match them to the labels.

In [ ]:
# Dataframe with file content and labels
class_df = pd.DataFrame(columns=['content', 'cik', 'year', 'quarter', 'subsector', 'name', 'tic',
                                 'SALES_value', 'OIADP', 'AT', 'CEQ', 'SALES', 'INTAN', 'ROE', 'OIADP_change',
                                 'AT_change', 'CEQ_change', 'SALES_change', 'INTAN_change', 'ROE_change'])

# Iterate through data directory
for path in glob.iglob(REPORT_PATH):
    file_name = path.split('/')[-1]
    year = sec_regex.search(file_name).group(1)
    quarter = sec_regex.search(file_name).group(2)
    cik = sec_regex.search(file_name).group(3)

    # Match file with label
    label_row = fin_label_df.loc[(fin_label_df['cik'] == int(cik)) & \
                             (fin_label_df['year'] == int(year)) & \
                             (fin_label_df['quarter'] == int(quarter))]
    if len(label_row) > 0:
        # Add cleaned file content and labels to dataframe
        with open(path, 'r') as file:
            class_df.loc[len(class_df)] = [file.read(),
                                           int(label_row.iloc[0]['cik']),
                                           int(label_row.iloc[0]['year']),
                                           int(label_row.iloc[0]['quarter']),
                                           label_row.iloc[0]['subsector'],
                                           label_row.iloc[0]['name'],
                                           label_row.iloc[0]['tic'],
                                           label_row.iloc[0]['saleq'],
                                           label_row.iloc[0]['oiadpq_posneg'],
                                           label_row.iloc[0]['atq_posneg'],
                                           label_row.iloc[0]['ceqq_posneg'],
                                           label_row.iloc[0]['saleq_posneg'],
                                           label_row.iloc[0]['intanq_posneg'],
                                           label_row.iloc[0]['ROE_posneg'],
                                           label_row.iloc[0]['oiadpq_change'],
                                           label_row.iloc[0]['atq_change'],
                                           label_row.iloc[0]['ceqq_change'],
                                           label_row.iloc[0]['saleq_change'],
                                           label_row.iloc[0]['intanq_change'],
                                           label_row.iloc[0]['ROE_change']]

print('Number of reports matched: %d' % len(class_df))
class_df.head()

Number of reports matched: 4068


,content,cik,year,quarter,subsector,name,tic,SALES_value,OIADP,AT,CEQ,SALES,INTAN,ROE,OIADP_change,AT_change,CEQ_change,SALES_change,INTAN_change,ROE_change
0,Item7.Management's Discussion and Analysis of ...,855654.0,2012.0,3.0,35201010.0,IMMUNOGEN INC,IMGN,3.252,pos,neg,neg,neg,NA,pos,19.308,-8.160,-16.883,-9.225,0.000,43.542
1,The following discussion and analysis provide...,788920.0,2011.0,3.0,35101010.0,PRO-DEX INC/CO,PDEX,6.876,neg,pos,pos,neg,NA,neg,-14.519,6.611,11.238,-1.891,0.000,-23.155
2,Item 7. Managements Discussion and Analysis of...,795551.0,2011.0,1.0,35101010.0,THERAGENICS CORP,TGX,20.253,pos,pos,pos,pos,neg,pos,50.771,1.415,1.731,6.335,-6.162,48.205
3,Managements Discussion and Analysis of Financ...,352915.0,2012.0,1.0,35102020.0,UNIVERSAL HEALTH SVCS INC,UHS,1755.536,neg,neg,pos,neg,neg,neg,-4.086,-0.316,4.514,-1.882,-0.727,-8.229
4,Item7.\n\nManagements Discussion and Analysis ...,1027915.0,2010.0,1.0,NaN,None,None,14.887,pos,pos,pos,pos,neg,neg,16.123,3.150,16.311,5.018,-0.296,-0.162


In [ ]:
# Clean up the data
class_df['content'] = cleanup_mda(class_df['content'])
class_df.head()

In [ ]:
# Save data as Pickle
with open(PICKLE_FILE, 'wb') as f:
    pickle.dump(class_df, f, pickle.HIGHEST_PROTOCOL)